In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import StandardScaler

# Custom dataset class
class TimeSeriesImageDataset(Dataset):
    def __init__(self, numerical_data, image_dir, seq_length=24, transform=None):
        self.numerical_data = numerical_data
        self.image_dir = image_dir
        self.seq_length = seq_length
        self.transform = transform
        self.scaler = StandardScaler()
        self.numerical_data.iloc[:, 2:] = self.scaler.fit_transform(self.numerical_data.iloc[:, 2:])
        
    def __len__(self):
        return len(self.numerical_data) - self.seq_length

    def __getitem__(self, idx):
        num_seq = self.numerical_data.iloc[idx:idx + self.seq_length, 2:].values.astype(np.float32)
        date_hour_seq = self.numerical_data.iloc[idx:idx + self.seq_length, :2]
        images = []

        for _, row in date_hour_seq.iterrows():
            date_str = row['date'].strftime('%Y%m%d')
            hour = row['hour']
            image_path = os.path.join(self.image_dir, date_str, f"{hour}.jpg")
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            images.append(image)
        
        images = torch.stack(images)
        target = self.numerical_data.iloc[idx + self.seq_length, 2:].values.astype(np.float32)
        return {'numerical': torch.tensor(num_seq), 'images': images, 'target': torch.tensor(target)}

# Load numerical data
numerical_data = pd.read_csv('./dataset/Linkou_2022.csv')
numerical_data = numerical_data.drop(columns=['Station'])
numerical_data['date'] = pd.to_datetime(numerical_data['date'],format="%d-%m-%Y %H:%M",dayfirst=True)

# Melt the data frame to have hours as a column
numerical_data = numerical_data.melt(id_vars=['date', 'measurement'], var_name='hour', value_name='value')
numerical_data['hour'] = numerical_data['hour'].astype(int)

# Check for non-numeric values in 'value' column and convert to numeric
numerical_data['value'] = pd.to_numeric(numerical_data['value'], errors='coerce')
numerical_data['value'].fillna(0, inplace=True)

# Pivot the table
numerical_data = numerical_data.pivot_table(index=['date', 'hour'], columns='measurement', values='value').reset_index()

# Transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the dataset and dataloader
image_dir = './dataset/009_Linkou'
seq_length = 24

dataset = TimeSeriesImageDataset(numerical_data, image_dir, seq_length, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
print("About to get sample from dataloader...")
# Print a sample for verification
sample = next(iter(dataloader))
print(sample['numerical'].shape, sample['images'].shape, sample['target'].shape)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24068\3248041937.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  numerical_data['value'].fillna(0, inplace=True)


About to get sample from dataloader...


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultiModalModel(nn.Module):
    def __init__(self, cnn_output_size, lstm_hidden_size, seq_length, num_features, num_classes):
        super(MultiModalModel, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, cnn_output_size)
        
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=lstm_hidden_size, num_layers=1, batch_first=True)
        
        self.fc = nn.Linear(cnn_output_size + lstm_hidden_size, num_classes)

    def forward(self, numerical_data, images):
        # Process images through CNN
        batch_size, seq_length, c, h, w = images.size()
        cnn_output = []
        for t in range(seq_length):
            out = self.cnn(images[:, t])
            cnn_output.append(out)
        
        cnn_output = torch.stack(cnn_output, dim=1)

        # Process numerical data through LSTM
        lstm_output, _ = self.lstm(numerical_data)

        # Take the last output of the LSTM
        lstm_output = lstm_output[:, -1, :]

        # Combine outputs
        combined_output = torch.cat((cnn_output[:, -1, :], lstm_output), dim=1)

        # Fully connected layer
        output = self.fc(combined_output)
        
        return output

# Parameters
cnn_output_size = 512
lstm_hidden_size = 128
seq_length = 24
num_features = numerical_data.shape[1] - 2  # exclude 'date' and 'hour'
num_classes = num_features  # predicting all features for the next time step

# Instantiate model
model = MultiModalModel(cnn_output_size, lstm_hidden_size, seq_length, num_features, num_classes)
model = model.cuda()  # Move model to GPU

# Print model summary for verification
print(model)


In [ ]:
import torch.optim as optim

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
        numerical_data = batch['numerical'].cuda()
        images = batch['images'].cuda()
        targets = batch['target'].cuda()
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(numerical_data, images)
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
